# Fitting Description

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/digital_breakthrough/task_3

/content/drive/MyDrive/digital_breakthrough/task_3


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 72 kB 621 kB/s 
     |████████████████████████████████| 72 kB 951 kB/s 
     |████████████████████████████████| 8.6 MB 52.3 MB/s 
     |████████████████████████████████| 2.6 MB 56.7 MB/s 
     |████████████████████████████████| 74 kB 3.3 MB/s 
     |████████████████████████████████| 37.1 MB 79 kB/s 
     |████████████████████████████████| 948 kB 51.5 MB/s 
     |████████████████████████████████| 56 kB 4.0 MB/s 
     |████████████████████████████████| 392 kB 63.8 MB/s 
     |████████████████████████████████| 428 kB 65.6 MB/s 
     |████████████████████████████████| 112 kB 60.7 MB/s 
     |████████████████████████████████| 68 kB 7.4 MB/s 
     |████████████████████████████████| 77 kB 7.0 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 636 kB 58.0 MB/s 
     |████████████████████████████████| 3.3 MB 56.0 MB/s 
     |████████████████████████████████| 895 kB 51.4 MB/s 
     |███████████████████████

In [ ]:
cd /content/drive/MyDrive/digital_breakthrough/task_3

/content/drive/MyDrive/digital_breakthrough/task_3


## Load Data

In [ ]:
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
sys.path.append('.')
from definitions import DATA_PATH, TRAIN_IMAGES, DOWNLOADED_TRAIN_IMAGES
from airotica.utils import read_image

In [ ]:
train = pd.read_csv(DATA_PATH / 'train.csv')
train_url_only = pd.read_csv(DATA_PATH / 'train_url_only.csv')
train_url_loaded_images = pd.read_csv(DATA_PATH / 'train_loaded_images.csv')
test = pd.read_csv(DATA_PATH / 'test.csv')
sample_submission = pd.read_csv(DATA_PATH / 'sample_submission.csv')

In [ ]:
import os
from os import listdir
train_images = listdir(TRAIN_IMAGES)
guid_train_images = [f.split('.')[0] for f in train_images if os.path.getsize(f"{str(TRAIN_IMAGES / f)}") != 0]

In [ ]:
dummy = test[test.guid.isin(guid_train_images)]
test_only_description = test[~(test.guid.isin(guid_train_images))]
test_only_images = dummy[dummy.description.isna()]
test_images_and_description = dummy[~(dummy.description.isna())]
print('without image or description:', len(test_only_description[test_only_description.description.isna()]))
print('only description:', test_only_description.shape[0])
print('only images:', test_only_images.shape[0])
print('images and description:', test_images_and_description.shape[0])

without image or description: 0
only description: 106
only images: 547
images and description: 570


 ## FitImages

### Model

In [ ]:
import os
import sys
import torch
import yaml
import logging
import numpy as np
import random
from tqdm import tqdm
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

sys.path.append('.')
from definitions import ROOT_DIR
from airotica.utils import save_checkpoint, convert_dict_to_tuple
from airotica.models import load_model
from airotica.loss_function import get_loss
from airotica.optimizers import (
    get_optimizer,
    get_scheduler
)
from airotica.dataset import get_data_loaders
from airotica.train import train as train_model, validation

%matplotlib inline
%load_ext autoreload
%autoreload 2

log = logging.getLogger(__name__)

{"asctime": "2021-08-05 13:24:36", "name": "matplotlib.pyplot", "filename": "pyplot.py", "levelname": "DEBUG", "message": "Loaded backend module://ipykernel.pylab.backend_inline version unknown."}


In [ ]:
train_labels = train.typology.unique()
typology_to_label = dict(zip(sorted(train_labels), range(len(train_labels))))

In [ ]:
DATA_PATH = ROOT_DIR / 'data'
CONFIG_PATH = ROOT_DIR / 'config5.yml'

In [ ]:
with open(CONFIG_PATH) as f:
    data = yaml.safe_load(f)
config = convert_dict_to_tuple(dictionary=data)

device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)
print(f'device: {device_name}')

device: cuda


In [ ]:
seed = config.dataset.seed
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

os.environ['CUDA_VISIBLE_DEVICES'] = config.cuda_id

In [ ]:
failed_guid = ['5c989b07-c6a2-4c7b-bd48-e6f9bca7eb27', 
               'cbd73791-e011-40e1-8cc7-f72a745bcebb', 
               'bebd7545-b84c-4af1-84bd-425e2b6313a9',
               'baa188a2-d630-489f-b92e-6cfaba762b0b',
               '4c39c840-cf9f-4363-82bf-74ea084c4f0d',
               '94c866c7-6230-409f-a5f9-b50f976ab62f',
               '46ecbdac-521d-4967-8d04-78e1811d3403',
               '60cacca9-e773-4932-83ca-e4d61bf5c134',
               '76975b28-9dfd-4cb6-bb54-c10a71b886a8',
               'b8165817-1662-4c17-ab53-fd0525537140',
               '99d35800-949b-445f-a018-d2e60b67d9c6',
               '852f75be-19bf-4148-932c-1637facbe8bc',
               'b8165817-1662-4c17-ab53-fd0525537140',
               'a733aa01-0e72-4a69-9045-6d262110c766',
               '78d9b613-9ced-4c65-9cb0-6b2d4dc8006c',
               '26e283c4-5919-4586-912c-3a45bfe290da',
               '1e7d2f58-d625-415c-ab3e-68b13ac46649',
               '73480476-2ba5-4e2f-8a67-f37db72a3bf0',
               '5de42ac6-4652-4a01-998b-0e9a70b19485',
               'eaa4120d-9375-4215-baed-03d9eb71b1f8',
               '900f9bab-7440-4776-9974-dd1a62687609',
               '3bc9b4a8-e881-46c1-82ce-74045856f3f4',
               '56060482-90ab-4a34-816b-ff0c2d01d6f9',
               '36803a89-50a4-4856-916d-3fb0f8c2874a',
               '8eccaebb-9fef-4e89-b08f-0fbec5feada1',
               '593efb25-7798-45a8-a953-3e0160ab826d',
               'e80f1b44-3a97-424b-b27b-6c5d30934ec1',
               '8be3ede8-1da2-4d56-820a-fee889fc9b19',
               '2aaf7293-bf48-48ce-9924-5a3c8a5d5c12',
               'a8a34b22-cf29-4987-92dc-9615aa63c0d3',
               '90f467ca-061b-4bd1-9222-3ccef2b5a476',
               '9f675417-4575-43b1-8acd-affeeecf5a3d',
               '208199f2-d8a1-4bb1-ab53-ecb14829e3c7',
               '0d7cd8d8-61f5-44f7-a6e1-04a17f34eac6',
               '8363665f-118e-4be7-89e5-42d71effbdd4',
               'e8e87195-6840-4487-a4e8-e834318aae36',
               '51a9e383-813a-4c19-b32c-e3604b171972',
               '00ce78f0-fded-4dc3-949d-3c6a55e9a4c7',
               '500dc4d-371d-4bd2-a1cd-3300991cf237',
               '63c1ee48-4f17-4f63-9cce-df8f6d78bb7c',
               'c073a415-66dd-460e-84b7-8c1a31e2f21e',
               'f500dc4d-371d-4bd2-a1cd-3300991cf237',
               'd9a80d93-4c54-4327-bef3-9191ea18c4f1',
               '1e6a7d50-1472-4b42-94fe-f762832424ae',
               '25d36f15-5b50-4f82-98b4-2142dde916e9',
               'edd9f097-e93b-4b09-8725-ce96a9dace7c',
               '2cd86089-d749-4a3c-8ed2-6de7eceffd6d',
               'e5220363-2b9b-46f3-a0a8-128d47d21aa0']

### Data

In [ ]:
trained = pd.read_csv('./data/full_images_data_2.csv')

In [ ]:
label_to_typology = {v: k for k, v in typology_to_label.items()}

In [ ]:
for i in range(15):
    globals()[f'label_{i}'] = trained[trained['label'] == i]
    print(globals()[f'label_{i}'].shape[0], label_to_typology[i], i)

{"asctime": "2021-08-05 13:25:18", "name": "numexpr.utils", "filename": "utils.py", "levelname": "INFO", "message": "NumExpr defaulting to 4 threads."}
14027 графика 0
24186 документы 1
3365 живопись 2
695 оружие 3
22153 предметы археологии 4
4074 предметы естественнонаучной коллекции 5
1431 предметы минералогической коллекции 6
21596 предметы нумизматики 7
19836 предметы печатной продукции 8
21262 предметы прикладного искусства, быта и этнографии 9
1532 предметы техники 10
0 прочие 11
7746 редкие книги 12
505 скульптура 13
28772 фотографии и негативы 14


In [ ]:
full = pd.concat(
    [
     label_3,
     label_4,
     label_6,
     label_10,
     label_13,
     label_0,
     label_1,
     label_2,
     label_5,
     label_7,
     label_8,
     label_9,
     label_11,
     label_12,
     label_14
    ]
)

In [ ]:
mapper = {0:0,
          1:1,
          2:2,
          3:3,
          4:4,
          5:5,
          6:6,
          7:7,
          8:8,
          9:9,
          10:10,
          12:11,
          13:12,
          14:13
          }

In [ ]:
full['label'] = full['label'].map(mapper)

In [ ]:
# downloaded_train_images = listdir(DOWNLOADED_TRAIN_IMAGES)
# guid_downloaded_train_images = [f.split('.')[0] for f in downloaded_train_images 
#                                 if os.path.getsize(f"{str(DOWNLOADED_TRAIN_IMAGES / f)}") > 100]

In [ ]:
full_train_2 = (
    full[~(full.guid.isin(failed_guid))]
    .reset_index(drop=True)
)

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold

kf = KFold(n_splits=3, random_state=seed, shuffle=True)

In [ ]:
for i, (train_ind, test_ind) in enumerate(kf.split(full_train_2, full_train_2.label), 1):
    X_train, X_valid = full_train_2.iloc[train_ind], full_train_2.iloc[test_ind]
    print(f'train: {X_train.shape[0]}, valid: {X_valid.shape[0]}')
    print(X_train.label.unique(), X_valid.label.unique())

    print("Loading model...")
    net = load_model(config, 
                     device=device_name)
    print("Done.")
    criterion, criterion_val = get_loss(config, device=device_name)
    optimizer = get_optimizer(config, 'SGD', net)

    n_epoch = 13
    scheduler = get_scheduler(config, optimizer)
    train_epoch = tqdm(range(config.train.n_epoch),
                    dynamic_ncols=True,
                    desc='Epochs',
                    position=0)

    dt, dv = get_data_loaders(train_data=X_train, 
                              valid_data=X_valid, 
                              config=config, 
                              img_path=TRAIN_IMAGES, 
                              download_img_path=DOWNLOADED_TRAIN_IMAGES, 
                              use_data=True)

    out_dir = str(ROOT_DIR / os.path.join(config.outdir, f'full_fold_{i}'))
    print("Savedir: {}".format(out_dir))
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    f1_best = 0
    for epoch in train_epoch:
        train_model(net, dt, criterion, optimizer, config, epoch)
        f1_val = validation(net, dv, criterion_val, epoch)
        if f1_val > f1_best:
            save_checkpoint(net, optimizer, scheduler, epoch, out_dir, f1_val)
            f1_best = f1_val
        scheduler.step()

train: 114089, valid: 57045
[ 3  4  6 10 12  0  1  2  5  7  8  9 11 13] [ 3  4  6 10 12  0  1  2  5  7  8  9 11 13]
Loading model...
{"asctime": "2021-08-05 13:26:26", "name": "airotica.models", "filename": "models.py", "levelname": "INFO", "message": "ResNext101_32x8d"}


Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-8ba56ff5.pth


  0%|          | 0.00/340M [00:00<?, ?B/s]

Done.
{"asctime": "2021-08-05 13:26:39", "name": "airotica.optimizers", "filename": "optimizers.py", "levelname": "INFO", "message": "0.002"}
{"asctime": "2021-08-05 13:26:39", "name": "airotica.optimizers", "filename": "optimizers.py", "levelname": "INFO", "message": "Opt: SGD"}


Epochs:   0%|          | 0/21 [00:00<?, ?it/s]

...


Train: 100%|██████████| 2376/2376 [2:17:09<00:00,  3.46s/it]


Val:   0%|          | 0/1189 [00:00<?, ?it/s]


    Train process of epoch: 2 is done;
    loss: 0.3594; acc: 0.90, f1: 0.90153310
    




Val:   0%|          | 1/1189 [00:39<13:01:04, 39.45s/it]

Val:   0%|          | 2/1189 [00:39<5:24:49, 16.42s/it] 

Val:   0%|          | 3/1189 [00:40<2:59:02,  9.06s/it]

Val:   0%|          | 4/1189 [00:40<1:50:35,  5.60s/it]

Val:   0%|          | 5/1189 [00:40<1:12:53,  3.69s/it]

Val:   1%|          | 6/1189 [00:40<50:04,  2.54s/it]  

Val:   1%|          | 7/1189 [00:41<35:35,  1.81s/it]

Val:   1%|          | 8/1189 [00:41<26:07,  1.33s/it]

Val:   1%|          | 9/1189 [00:43<29:16,  1.49s/it]

Val:   1%|          | 10/1189 [00:43<22:01,  1.12s/it]

Val:   1%|          | 11/1189 [00:43<17:04,  1.15it/s]

Val:   1%|          | 12/1189 [00:44<14:07,  1.39it/s]

Val:   1%|          | 13/1189 [00:44<11:35,  1.69it/s]

Val:   1%|          | 14/1189 [00:44<09:52,  1.98it/s]

Val:   1%|▏         | 15/1189 [00:45<08:43,  2.24it/s]

Val:   1%|▏         | 16/1189 [00:45<07:50,  2.49it/s]

Val:   1%|▏         | 17/1189 [01:14<2:54:29,  8.93s/it]

Val:   2%|▏         | 18/1189 [01:14<2:


        Validation of epoch: 2 is done; 
        loss: 0.5485; acc: 0.83, f1: 0.90175904
        


Train:   1%|▏         | 32/2376 [06:53<52:27,  1.34s/it]

In [ ]:
https://drive.google.com/file/d/1g5FTydmkt5d5nNWWYeFroM3zsA6wlVaM/view?usp=sharing

In [ ]:
https://github.com/pytorch/pytorch/issues/1137